## Cython

<https://doc.sagemath.org/html/en/thematic_tutorials/cython_interface.html>

<https://doc.sagemath.org/html/en/developer/coding_in_cython.html>

In [ ]:
# Ignore this; my laptop is so old and Apple is nasty so I need this
sdkroot = !xcrun --show-sdk-path
%env SDKROOT={sdkroot[0]}

env: SDKROOT=/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk


In [1]:
%%bash
brew install pkg-config

bash: line 1: brew: command not found


CalledProcessError: Command 'b'brew install pkg-config\n'' returned non-zero exit status 127.

In [ ]:
%%bash
cat > hello.c << CCODE

#include <stdio.h>
void hello_world(){
    printf("Hello World\n");
}

CCODE

In [ ]:
%%cython

cdef extern from "hello.c":
    void hello_world()

def my_bridge_function():
    hello_world() # This is the C function from hello.c

RuntimeError: command '/usr/bin/clang' failed with exit code 1
clang: error: SDK "$(xcrun --show-sdk-path)" cannot be located
In file included from /var/folders/3p/r_29jkjx76b9g_qktgdtf6_w0000gn/T/sage_ffhlhs6u/spyx_uuvq9gy1/_var_folders_3p_r_29jkjx76b9g_qktgdtf6_w0000gn_T_sage_ffhlhs6u_tmp_jafsmbe9_pyx/_var_folders_3p_r_29jkjx76b9g_qktgdtf6_w0000gn_T_sage_ffhlhs6u_tmp_jafsmbe9_pyx_0.c:46:
/private/var/tmp/sage-10.6-current/local/var/lib/sage/venv-python3.12.5/include/python3.12/Python.h:23:12: fatal error: 'stdlib.h' file not found
#  include <stdlib.h>
           ^~~~~~~~~~
1 error generated.

In [ ]:
%%bash
cat > ispow2.spyx << CYTHONCODE

def is2pow(n):
    while n != 0 and n%2 == 0:
        n = n >> 1
    return n == 1

CYTHONCODE

In [ ]:
# set up a sane toolchain for this session
%env CC=clang
%env CXX=clang++
%env SDKROOT=$(xcrun --show-sdk-path)
%env LDFLAGS=
%env CFLAGS=-mmacosx-version-min=12.0
%env LDSHARED="clang -bundle -undefined dynamic_lookup"

env: CC=clang
env: CXX=clang++
env: SDKROOT=$(xcrun --show-sdk-path)
env: LDFLAGS=
env: CFLAGS=-mmacosx-version-min=12.0
env: LDSHARED="clang -bundle -undefined dynamic_lookup"


In [ ]:
load('ispow2.spyx')

Compiling ispow2.spyx...


RuntimeError: command '/usr/bin/clang' failed with exit code 1
clang: error: SDK "$(xcrun --show-sdk-path)" cannot be located
In file included from /var/folders/3p/r_29jkjx76b9g_qktgdtf6_w0000gn/T/sage_ffhlhs6u/spyx_uuvq9gy1/_Users_ThinBold_Git_ccpppy2025_slides_1112_ispow2_spyx/_Users_ThinBold_Git_ccpppy2025_slides_1112_ispow2_spyx_4.c:45:
/private/var/tmp/sage-10.6-current/local/var/lib/sage/venv-python3.12.5/include/python3.12/Python.h:23:12: fatal error: 'stdlib.h' file not found
#  include <stdlib.h>
           ^~~~~~~~~~
1 error generated.

In [ ]:
attach("ispow2.spyx")

AttributeError: 'SageZMQInteractiveShell' object has no attribute '_inputhook'